<a href="https://colab.research.google.com/github/louisdennington-design/decision-tree-dissertation/blob/main/html_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Mount Google Drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import re

In [4]:
# Global parameters

URL_abortion = 'https://www.nice.org.uk/guidance/ng140/chapter/Recommendations'
URL_pneumonia = 'https://www.nice.org.uk/guidance/ng250/chapter/Recommendations'
URL_bipolar = 'https://www.nice.org.uk/guidance/cg185/chapter/Recommendations'

# regex indicating start of relevant text?
# regex indicating end of relevant text?

In [ ]:
# Needs incorporation into functions below or deletion?

from dataclasses import dataclass

@dataclass
class recommendation_entity:
    """
    Defines the format that text is stored in when scraped from the html
    """
    recommendation_id: int
    parent_heading: str
    recommendation_text: str
    recommendation_url: str # Sometimes a bullet point incorporates a hyperlink to refer to earlier guidance

# If needing to force a finite list in one of the class fields

from enum import Enum

class types_of_thing(Enum):
    """
    1. Adjust the name
    2. Add it as a field, e.g., types_of_thing: types_of_thing
    """
    THIS = "this"
    THAT = "that"
    OR_THIS = "or_this"

In [5]:
# Scrape the text

from bs4 import BeautifulSoup
import requests

url = URL_bipolar

html_scrape = requests.get(url).text

soup = BeautifulSoup(html_scrape, "html.parser")

#print(soup.prettify())

In [6]:
# Processing function


# FIXES NEEDED
## What is being labelled as 'sub_heading_2' should instead be 'recommendation_text' or similar
## Headings before bullet points are being omitted and only the bullet points saved, e.g., "Give carers written and verbal information in an accessible format about:""
## Sections 1.3 and 1.4 have been skipped, others too - Why?
## Box on Valproate (<div class="panel panel--primary">) should be included
## Check how / whether headings of boxes ("strong") and some sub-sections (e.g., "Starting antipsychotic medication") are being ingested properly

"""
- begin at <div class="section" title="1.1
- end before "Terms used in this guideline"
- store every <div class="recommendation__body">
- store the parent heading <h3 class="title" id="">, sub-heading (if exists) <h4 class="title" id="">, and sub-sub-heading <h5 class="title" id=""
- collapse bullet points into recommendation:

        <ul class="itemizedlist indented">
            <li class="listitem">

- exclude <div class="panel">
- save <div class="panel panel--primary"> as separate text to be accessed later if needed
- if link occurs within <div class="section">, as <a class="link" href="", then extract the URL

- if doing the Pneumonia guideline, will also need to think about how to digest <div class="informaltable">
"""

def process_html(soup):

    raw_guideline_data = []
    current_heading = None
    sub_heading_1 = None

    start = soup.find("h3", string = lambda s: s and s.strip().startswith("1.1"))
    if start is None:
        print("Start point specified in function does not match page formatting.")
        return raw_guideline_data

    current_heading = start.get_text(strip=True)

    for element in start.next_elements:
        if getattr(element, "name", None) is None:
            continue # so that it only recognises tags, and not unnamed strings

        classes = element.get("class", [])

        if element.name == "h3" and "title" in classes:
            current_heading = element.get_text(strip=True)
            sub_heading_1 = None

        elif element.name == "h4" and "title" in classes:
            sub_heading_1 = element.get_text(strip=True)

        elif element.name == "article" and "recommendation" in classes:
            number_tag = element.find("h5", class_="recommendation__number")
            body_tag = element.find("div", class_="recommendation__body")

            if not number_tag or not body_tag:
                continue

            sub_heading_2 = number_tag.get_text(strip=True)

            bullet_points = []
            for li in body_tag.find_all("li"):
                bullet_points.append(li.get_text(" ", strip=True).replace("\xa0", " "))

            if bullet_points:
                recommendation_text = ", and ".join(bullet_points)
            else:
                recommendation_text = body_tag.get_text(" ", strip=True)

            recommendation_text = recommendation_text.replace("\xa0", " ")

            raw_guideline_data.append({
                "heading_1": current_heading,
                "sub_heading_1": sub_heading_1,
                "sub_heading_2": sub_heading_2,
                "text": recommendation_text})

    return raw_guideline_data

raw_guideline_data = process_html(soup)

print(raw_guideline_data)

[{'heading_1': '1.1 Care for adults, children and young people across all phases of bipolar disorder', 'sub_heading_1': 'Treatment and support for specific populations', 'sub_heading_2': '1.1.1', 'text': 'Ensure that older people with bipolar disorder are offered the same range of treatments and services as younger people with bipolar disorder. [2014]'}, {'heading_1': '1.1 Care for adults, children and young people across all phases of bipolar disorder', 'sub_heading_1': 'Treatment and support for specific populations', 'sub_heading_2': '1.1.2', 'text': "Offer people with bipolar disorder and coexisting disorders, such as personality disorder, attention deficit hyperactivity disorder, anxiety disorders or substance misuse, treatment in line with the relevant NICE guideline, in addition to their treatment for bipolar disorder. See NICE's guidelines on antisocial personality disorder , borderline personality disorder , attention deficit hyperactivity disorder , generalised anxiety disord